In [1]:
import pyam
import pandas as pd
import numpy as np

import dotenv
import os
from pathlib import Path

<IPython.core.display.Javascript object>

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
dotenv.load_dotenv()

True

<IPython.core.display.Javascript object>

Step 1: Read in the global AR6 dataset and downselect for the following potential filters:
* Primary Energy|Biomass
* Land Cover|Cropland|Energy Crops

In [4]:
df = pd.read_csv(
    os.environ['AR6_DATA'],
    index_col=[0,1]
)

<IPython.core.display.Javascript object>

In [5]:
df_bio_pe = (
    df[df['Variable']=='Primary Energy|Biomass']
)

<IPython.core.display.Javascript object>

In [6]:
df_land_cover = (
    df[df['Variable']=='Land Cover|Cropland|Energy Crops']
)

<IPython.core.display.Javascript object>

Step 2: Read in the scenarios that are covered by Gidden et al.

In [7]:
gidden_scens = pd.read_excel(
    os.environ['GIDDEN_DATA'],
    sheet_name='meta',
    index_col=[0,1]
)

<IPython.core.display.Javascript object>

Step 3: Pull out the intersection of the scenarios.

In [8]:
scens_intersection = (
    df_bio_pe
    .index
    .intersection(gidden_scens.index)
)

<IPython.core.display.Javascript object>

In [9]:
df_bio_pe_filtered = (
    df_bio_pe
    .loc[scens_intersection]
)

<IPython.core.display.Javascript object>

In [10]:
df_bio_pe_filtered_pyam = pyam.IamDataFrame(
    df_bio_pe_filtered
    .reset_index()
)

<IPython.core.display.Javascript object>

In [11]:
scens_intersection_land = (
    df_land_cover
    .index
    .intersection(gidden_scens.index)
)

<IPython.core.display.Javascript object>

In [12]:
df_land_cover_filtered = pyam.IamDataFrame(
    df_land_cover
    .loc[scens_intersection_land]
    .reset_index()
)

<IPython.core.display.Javascript object>

Step 4: Identify the maximum level of PE bioenergy as well as the maximum land cover allocated to energy crops. Do the same at the global year of net zero CO2.

In [13]:
df_bio_pe_filtered_pyam.set_meta_from_data(
    name='pe_bio_max',
    variable='Primary Energy|Biomass',
    method=np.max
)

<IPython.core.display.Javascript object>

In [14]:
gidden_scens.loc[:, 'Year of netzero CO2 emissions (Harm-Infilled) table'].fillna(2100, inplace=True)

<IPython.core.display.Javascript object>

In [15]:
df_bio_pe_filtered_pyam.set_meta(
    (
        df_bio_pe_filtered_pyam
        .interpolate(
            time=range(2020, 2101)
        )
        .filter(
            variable='Primary Energy|Biomass'
        )
        .timeseries()
        .apply(
            lambda x: x[gidden_scens.loc[x.name[0:2], 'Year of netzero CO2 emissions (Harm-Infilled) table']],
            axis=1
        )
    ),
        name='pe_bio_netzero'
)

<IPython.core.display.Javascript object>

In [16]:
df_land_cover_filtered.set_meta_from_data(
    name='land_cover_max',
    variable='Land Cover|Cropland|Energy Crops',
    method=np.max
)

<IPython.core.display.Javascript object>

In [17]:
df_land_cover_filtered.set_meta(
    (
        df_land_cover_filtered
        .interpolate(
            time=range(2020, 2101)
        )
        .filter(
            variable='Land Cover|Cropland|Energy Crops'
        )
        .timeseries()
        .apply(
            lambda x: x[gidden_scens.loc[x.name[0:2], 'Year of netzero CO2 emissions (Harm-Infilled) table']],
            axis=1
        )
    ),
        name='land_cover_netzero'
)

<IPython.core.display.Javascript object>

Step 5: Set a new exclude column to False.

In [18]:
df_bio_pe_filtered_pyam.set_meta(
    name='bio_sustainable',
    meta=False
)

<IPython.core.display.Javascript object>

In [19]:
df_land_cover_filtered.set_meta(
    name='land_sustainable',
    meta=False
)

<IPython.core.display.Javascript object>

In [20]:
scens_pass = (
    df_bio_pe_filtered_pyam.meta[df_bio_pe_filtered_pyam.meta['pe_bio_max'] <= 100].index
)

<IPython.core.display.Javascript object>

In [21]:
df_bio_pe_filtered_pyam.set_meta(
    name='bio_sustainable',
    meta=True,
    index=scens_pass
)

<IPython.core.display.Javascript object>

In [22]:
filter_set_bio = df_bio_pe_filtered_pyam.meta

<IPython.core.display.Javascript object>

In [23]:
filter_set_bio.to_csv(
    Path(
        '../data/103_filter_set_bio.csv'
    )
)

<IPython.core.display.Javascript object>

In [24]:
df_land_cover_filtered.meta.to_csv(
    Path(
        '../data/103_filter_set_land.csv'
    )
)

<IPython.core.display.Javascript object>

In [25]:
df_combined = pyam.concat(
    [
        df_bio_pe_filtered_pyam,
        df_land_cover_filtered
    ]
)

/Users/gauravganti/opt/anaconda3/envs/ar6_cdr_assessment/lib/python3.10/site-packages/pyam/utils.py:411: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  left = pd.concat([left, right.loc[diff, :]], sort=False)


<IPython.core.display.Javascript object>

In [26]:
df_combined.to_csv(
    Path(
        '../data/103_combined_data.csv'
    )
)

<IPython.core.display.Javascript object>